In [3]:
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('/home/matt/readout/host/')
import ali_offline_demod as dm
import imp
import glob
import pickle
from scipy import spatial
import h5py
from scipy.fft import fft, ifft, fftfreq
from copy import deepcopy
from scipy.interpolate import CubicSpline, interp1d
from scipy.optimize import curve_fit
from tqdm.notebook import tqdm
import imp
import pandas as pd

In [72]:
imp.reload(dm)
ts_file = 'ts_toneinit_fcenter_4250.0_20240530105822_t_20240530113414.hd5'
f_sawtooth = 15.26

processed = dm.full_demod_process(ts_file, 
                                  f_sawtooth, 
                                  method='fft', 
                                  n=0, 
                                  channels='all',
                                  start_channel=259,
                                  stop_channel=261,
                                  tone_init_path = '/home/matt/ali_drive_mnt/tone_initializations', 
                                  ts_path = '/home/matt/ali_drive_mnt/time_streams',
                                  display_mode = 'notebook')    

4250.0
20240530105822
/home/matt/ali_drive_mnt/tone_initializations/fcenter_4250.0_20240530105822/
num of channels: 1002
num of tones: 233
looking for delay region


/home/matt/readout/host/ali_offline_demod.py:998: RuntimeWarning: divide by zero encountered in log10
  


start = 4022797851.5625
stop = 4032818359.375
shape_cals: (233, 3)
2.4613431293477186
n_phi0: 2.4613431293477186
start index: 16


  0%|          | 0/233 [00:00<?, ?it/s]

In [8]:
%matplotlib qt
plt.plot(processed['fr t'], processed['fr data'][10])

In [42]:
ts_file = 'ts_toneinit_fcenter_4250.0_20240531182749_t_20240531183944.hd5'
f_sawtooth = 15.26

processed2 = dm.full_demod_process(ts_file, 
                                  f_sawtooth, 
                                  method='fft', 
                                  n=0, 
                                  channels='all',
                                  start_channel=259,
                                  stop_channel=261,
                                  tone_init_path = '/home/matt/ali_drive_mnt/tone_initializations', 
                                  ts_path = '/home/matt/ali_drive_mnt/time_streams',
                                  display_mode = 'notebook')    

4250.0
20240531182749
/home/matt/ali_drive_mnt/tone_initializations/fcenter_4250.0_20240531182749/
num of channels: 1002
num of tones: 65
looking for delay region
start = 4026304687.5
stop = 4036325195.3125
1.7776367045289059
n_phi0: 1.7776367045289059
start index: 18


  0%|          | 0/65 [00:00<?, ?it/s]

In [37]:
data = processed2['fr data'][20]

derivative = np.diff(data)
reset = np.argmax(derivative[0:31])

data_drop = data[reset+1:]
end = int(len(data_drop)/256)*256
data_drop = data_drop[:end]

data_matrix = np.reshape(data_drop, (int(len(data_drop)/256),256))

print(data_matrix.shape)


subtract = np.array([data_matrix[i,:] - data_matrix[i-1,:] for i in np.arange(1,len(data_matrix),1)])

#print(subtract)

derivative_matrix = np.diff(data_matrix)

#print(subtract.shape)
#print(derivative_matrix.shape)

phi_matrix_naive = subtract[:,1:] / derivative_matrix[:-1,:]

phi_naive = np.array([np.average(row) for row in phi_matrix_naive])


phi_matrix_weighted = subtract[:,1:] * derivative_matrix[:-1,:]

phi_weighted = np.array([np.average(row) for row in phi_matrix_weighted])

fig_result, ax_result = plt.subplots(1)
ax_result.plot(phi_weighted)

(304, 256)


/tmp/ipykernel_162939/447726887.py:24: RuntimeWarning: divide by zero encountered in divide
  phi_matrix_naive = subtract[:,1:] / derivative_matrix[:-1,:]


In [43]:
%matplotlib qt
plt.plot(processed2['fr data'][20])

In [63]:
#looking at the frequencies associated with the master channel list
%matplotlib qt
imp.reload(dm)

#ts_file = 'ts_toneinit_fcenter_4250.0_20240216162215_t_20240216163432.hd5'
ts_file = 'ts_toneinit_fcenter_4250.0_20240604115424_t_20240604142439.hd5'
f_sawtooth = 15.26

processed = dm.full_demod_process(ts_file, 
                                  f_sawtooth, 
                                  method='fft', 
                                  n=0, 
                                  channels='all',
                                  start_channel=259,
                                  stop_channel=261,
                                  correct_phase_jumps=True,
                                  tone_init_path = '/home/matt/ali_drive_mnt/tone_initializations', 
                                  ts_path = '/home/matt/ali_drive_mnt/time_streams',
                                  display_mode = 'notebook')    

#plt.plot(processed['raw i'][1])
#plt.plot(processed['raw q'][1])
#plt.plot(processed['fr data'][1])

#print(processed['fr data'])

fig, ax = plt.subplots(1)
for i in range(len(processed['demod data'])):
    ax.plot(processed['demod t'], processed['demod data'][i] - np.average(processed['demod data'][i]) + 0.1*i)
    
ax.set_ylim([0,1])

4250.0
20240604115424
/home/matt/ali_drive_mnt/tone_initializations/fcenter_4250.0_20240604115424/
num of channels: 1002
num of tones: 231
looking for delay region


/home/matt/readout/host/ali_offline_demod.py:1147: RuntimeWarning: divide by zero encountered in log10
  y=np.append(y,20*np.log10(np.abs(test_sweep[1,:])))


start = 4018289062.5
stop = 4028309570.3125
delay: 3.437642115431055e-08
shape_cals: (231, 3)
[1645.81950038-2122.48711937j 1531.9952127    +0.j
    2.23763721   +0.j        ]
2.4613431293477186
n_phi0: 2.4613431293477186
index: 2313
len times: 4880
index: 1526
len times: 4880
index: 1478
len times: 4880
index: 1478
len times: 4880
index: 1478
len times: 4880
index: 175
len times: 4880
index: 1478
len times: 4880
index: 1526
len times: 4880
index: 1478
len times: 4880
index: 206
len times: 4880
index: 1478
len times: 4880
index: 4243
len times: 4880
index: 491
len times: 4880
index: 1478
len times: 4880
index: 767
len times: 4880
index: 1478
len times: 4880
index: 1478
len times: 4880
index: 4243
len times: 4880
index: 767
len times: 4880
index: 1478
len times: 4880
index: 1478
len times: 4880
index: 963
len times: 4880
index: 1478
len times: 4880
index: 4243
len times: 4880
index: 1478
len times: 4880
index: 175
len times: 4880
index: 767
len times: 4880
index: 206
len times: 4880
ind

  0%|          | 0/231 [00:00<?, ?it/s]

(0.0, 1.0)

In [71]:
frequencies = processed['channel freqs']
print(frequencies)
frequencies_df = pd.DataFrame({'freqs': np.real(frequencies)})
frequencies_df.to_csv('/home/matt/readout/host/channel_standards/raw_freq_lists/LO4750_freqs.csv',sep=',')

[4.50071096e+09+0.j 4.53463467e+09+0.j 4.53683017e+09+0.j
 4.53922670e+09+0.j 4.54085952e+09+0.j 4.54243188e+09+0.j
 4.54480155e+09+0.j 4.54699049e+09+0.j 4.54875381e+09+0.j
 4.55075893e+09+0.j 4.55291830e+09+0.j 4.55485940e+09+0.j
 4.55723928e+09+0.j 4.56014048e+09+0.j 4.56335614e+09+0.j
 4.56687972e+09+0.j 4.56869989e+09+0.j 4.57067662e+09+0.j
 4.57117476e+09+0.j 4.57311988e+09+0.j 4.57397877e+09+0.j
 4.57478444e+09+0.j 4.57683587e+09+0.j 4.57908041e+09+0.j
 4.58060516e+09+0.j 4.58267425e+09+0.j 4.58496794e+09+0.j
 4.58692552e+09+0.j 4.58906166e+09+0.j 4.59167511e+09+0.j
 4.59518069e+09+0.j 4.59648847e+09+0.j 4.59838209e+09+0.j
 4.60102444e+09+0.j 4.60238020e+09+0.j 4.60420184e+09+0.j
 4.60542473e+09+0.j 4.60785161e+09+0.j 4.60932705e+09+0.j
 4.61084282e+09+0.j 4.61311905e+09+0.j 4.61407413e+09+0.j
 4.61496942e+09+0.j 4.61636021e+09+0.j 4.62241997e+09+0.j
 4.62409649e+09+0.j 4.62602041e+09+0.j 4.62770810e+09+0.j
 4.63233182e+09+0.j 4.63443664e+09+0.j 4.63605680e+09+0.j
 4.63778499e+0

In [61]:
#/home/matt/ali_drive_mnt/time_streams/

#ts_file = 'ts_toneinit_fcenter_4250.0_20240604115424_t_20240604121938.hd5'
imp.reload(dm)
%matplotlib qt

ts_file = 'ts_toneinit_fcenter_4250.0_20240604115424_t_20240604140308.hd5' #internal flux ramp 10s
#ts_file = 'ts_toneinit_fcenter_4250.0_20240604115424_t_20240604142439.hd5' #internal flux ramp 10s
#ts_file = 'ts_toneinit_fcenter_4250.0_20240604115424_t_20240604164228.hd5' #external flux ramp 10s
#ts_file = 'ts_toneinit_fcenter_4250.0_20240604164841_t_20240604165339.hd5' #external flux ramp 10s
#ts_file = 'ts_toneinit_fcenter_4250.0_20240604164841_t_20240604170156.hd5' #2.5V FR
#ts_file = 'ts_toneinit_fcenter_4250.0_20240604164841_t_20240604170933.hd5'
#ts_file = 'ts_toneinit_fcenter_4250.0_20240604164841_t_20240604171346.hd5'
#ts_file = 'ts_toneinit_fcenter_4250.0_20240604172411_t_20240604172923.hd5'
#ts_file = 'ts_toneinit_fcenter_4250.0_20240216162215_t_20240216163432.hd5'
#ts_file = 'ts_toneinit_fcenter_4250.0_20240216162215_t_20240216163956.hd5'
#ts_file = 'ts_toneinit_fcenter_4250.0_20240604172411_t_20240604180256.hd5'
#ts_file = 'ts_toneinit_fcenter_4250.0_20240604172411_t_20240604183125.hd5'
#ts_file = 'ts_toneinit_fcenter_4250.0_20240603150744_t_20240603151428.hd5'
#ts_file = 'ts_toneinit_fcenter_4250.0_20240605135805_t_20240605140401.hd5' #hardware reverted to old fr setup
#ts_file = 'ts_toneinit_fcenter_4750.0_20240605141834_t_20240605142541.hd5'
#ts_file = 'ts_toneinit_fcenter_4750.0_20240605141834_t_20240605145707.hd5'
#ts_file = 'ts_toneinit_fcenter_4750.0_20240605141834_t_20240605150512.hd5'
#ts_file = 'ts_toneinit_fcenter_4750.0_20240605141834_t_20240605151107.hd5'
#ts_file = 'ts_toneinit_fcenter_4750.0_20240605151447_t_20240605152001.hd5'
#ts_file = 'ts_toneinit_fcenter_4750.0_20240605151447_t_20240605163547.hd5' #swapped sma cable for external fr
#ts_file = 'ts_toneinit_fcenter_4250.0_20240531181859_t_20240531182534.hd5'
#ts_file = 'ts_toneinit_fcenter_4450.0_20240528155454_t_20240528160657.hd5'
#ts_file = 'ts_toneinit_fcenter_4250.0_20240528175138_t_20240528175845.hd5'
#ts_file = 'ts_toneinit_fcenter_4250.0_20240513170334_t_20240513172359.hd5'
#ts_file = 'ts_toneinit_fcenter_4250.0_20240528175138_t_20240528175845.hd5'
#ts_file = 'ts_toneinit_fcenter_4450.0_20240528155454_t_20240528160142.hd5'
#ts_file = 'ts_toneinit_fcenter_4450.0_20240528155454_t_20240528160657.hd5'
#ts_file = 'ts_toneinit_fcenter_4450.0_20240523195528_t_20240523200703.hd5'
#ts_file = 'ts_toneinit_fcenter_4250.0_20240530135358_t_20240530140648.hd5'
#ts_file = 'ts_toneinit_fcenter_4250.0_20240506174818_t_20240506191017.hd5'


f_sawtooth = 15 #Hz

processed = dm.full_demod_process(ts_file, 
                                  f_sawtooth, 
                                  method='fft', 
                                  n=0, 
                                  channels='all',
                                  start_channel=259,
                                  stop_channel=261,
                                  tone_init_path = '/home/matt/ali_drive_mnt/tone_initializations', 
                                  ts_path = '/home/matt/ali_drive_mnt/time_streams',
                                  display_mode = 'notebook')    


fig, ax = plt.subplots(1)
for i in range(len(processed['demod data'])):
    ax.plot(processed['demod t'], processed['demod data'][i])

4250.0
20240604115424
/home/matt/ali_drive_mnt/tone_initializations/fcenter_4250.0_20240604115424/
num of channels: 1002
num of tones: 231
looking for delay region
start = 4018289062.5
stop = 4028309570.3125
delay: 3.437642115431055e-08
shape_cals: (231, 3)
[1645.81950038-2122.48711937j 1531.9952127    +0.j
    2.23763721   +0.j        ]
2.504006410256412
n_phi0: 2.504006410256412
index: 1825
len times: 4880
index: 812
len times: 4880
index: 1687
len times: 4880
index: 1687
len times: 4880
index: 1687
len times: 4880
index: 1526
len times: 4880
index: 1687
len times: 4880
index: 812
len times: 4880
index: 1687
len times: 4880
index: 1526
len times: 4880
index: 1687
len times: 4880
index: 3385
len times: 4880
index: 918
len times: 4880
index: 1687
len times: 4880
index: 2726
len times: 4880
index: 1687
len times: 4880
index: 1687
len times: 4880
index: 3385
len times: 4880
index: 2726
len times: 4880
index: 1687
len times: 4880
index: 1687
len times: 4880
index: 4485
len times: 4880
ind

  0%|          | 0/231 [00:00<?, ?it/s]

In [59]:
plt.plot(processed['demod t'], processed['demod data'][215])

In [36]:
%matplotlib qt
channel = 1
f_fr = 15

fr_t = processed['fr t']
fr_data = processed['fr data'][channel]
start_idx = processed['start index']

fig,ax = plt.subplots(1)
ax.plot(processed['fr t'], processed['fr data'][channel],'.-')
ax.plot(processed['fr t'][processed['start index']], processed['fr data'][channel][processed['start index']],'*')

t_fr_start = fr_t[start_idx:]
sig_fr_start = fr_data[start_idx:]

interpolation = interp1d(t_fr_start, sig_fr_start,fill_value='extrapolate')

t_final = round((t_fr_start[-1])/(1/f_fr))*(1/f_fr) #make final time the latest time that fits an integer number of flux ramps; will interpolate to here
t_start = t_fr_start[0]
t_elapsed = t_final - t_start
n_reset_periods = t_elapsed * f_fr #will be replaced when we have the fr reference

t_interp = np.linspace(t_start, t_final, round(n_reset_periods)*1024) #interpolate so that every chunk has 1024 points; will make fft faster        
sig_interp = interpolation(t_interp)




fig2, ax2 = plt.subplots(1)
ax2.plot(t_interp, sig_interp,'.')
ax2.set_xlabel('$t$ (s)')
ax2.set_ylabel('Resonator Position (arb.)')
#ax2.set_title(f'Ch. {ch}')

t_chunked = np.reshape(t_interp,(int(len(t_interp)/1024), 1024))
sig_chunked = np.reshape(sig_interp,(int(len(sig_interp)/1024), 1024))
sig_average = sig_chunked.mean(axis=1, keepdims=True)
sig_chunked = sig_chunked - sig_average

fig3, ax3 = plt.subplots(1)
for i in range(len(sig_chunked)):
    ax3.plot(t_chunked[i], sig_chunked[i])


"""

fig2, ax2 = plt.subplots(1)
ax2.plot(t_start, fr_start, '.-')

n_points = int(488.28 / freq_fr)

truncate_idx = int(int(len(fr_start) / (n_points)) * (n_points))

t_truncate = t_start[:truncate_idx] 
fr_truncate = fr_start[:truncate_idx]

fig3, ax3 = plt.subplots(1)
ax3.plot(t_truncate, fr_truncate, '.-')

t_chunk = np.reshape(t_truncate, (int(len(t_truncate)/n_points), n_points))
fr_chunk = np.reshape(fr_truncate, (int(len(fr_truncate)/n_points), n_points))

fig4, ax4 = plt.subplots(1)
for i in range(len(fr_chunk)):
    ax4.plot(t_chunk[i],fr_chunk[i])
    """

"\n\nfig2, ax2 = plt.subplots(1)\nax2.plot(t_start, fr_start, '.-')\n\nn_points = int(488.28 / freq_fr)\n\ntruncate_idx = int(int(len(fr_start) / (n_points)) * (n_points))\n\nt_truncate = t_start[:truncate_idx] \nfr_truncate = fr_start[:truncate_idx]\n\nfig3, ax3 = plt.subplots(1)\nax3.plot(t_truncate, fr_truncate, '.-')\n\nt_chunk = np.reshape(t_truncate, (int(len(t_truncate)/n_points), n_points))\nfr_chunk = np.reshape(fr_truncate, (int(len(fr_truncate)/n_points), n_points))\n\nfig4, ax4 = plt.subplots(1)\nfor i in range(len(fr_chunk)):\n    ax4.plot(t_chunk[i],fr_chunk[i])\n    "

In [64]:
%matplotlib qt
channel = 2
f_fr = 15

fr_data = processed['fr data'][channel]
start_idx = processed['start index']

fig,ax = plt.subplots(1)
ax.plot(fr_data,'.-')

data_new_start = fr_data[32:]
trunc_idx = int(len(data_new_start) / 32) * 32
data_truncated = data_new_start[:trunc_idx]

data_chunk = np.reshape(data_truncated, (int(len(data_truncated) / 32), 32))

fig2,ax2 = plt.subplots(1)
for i in range(len(data_chunk)):
    x = np.arange(0,32)
    ax2.plot(x+32*i, data_chunk[i],'.-')

In [4]:
print(processed['start index'])

17


In [104]:
imp.reload(dm)

fig, ax = plt.subplots(1)
for i in range(len(processed['fr data'])):
    t, data, reset = dm.demodulate_with_fft(t = processed['fr t'],
                                  sig = processed['fr data'][i],
                                  start_index = processed['start index'],
                                  f_fr = 15,
                                  phase_units='rad',
                                  correct_phase_jumps=False,
                                  phase_jump_threshold=0,
                                  plot_demod = False,
                                  plot_demod_title=None,
                                  intermediate_plotting_limits=[None,None],
                                  plot_chunking_process = False,
                                  plot_fft = False,
                                  plot_fft_no_dc = False,
                                  plot_limited_fft = False,
                                  plot_fit = False,
                                  plot_vectors = False)
    
    ax.plot(t, data - np.average(data) + i*0.1)


In [105]:
imp.reload(dm)

#t, data = dm.demodulate_with_fft_internal_fr(t = processed['fr t'],
#                                             sig = processed['fr data'][120],  #1 #110 #120
#                                             n_packets = 32, 
#                                             correct_phase_jumps = False, 
#                                             phase_jump_threshold=0.4)

reset_indices = [dm.find_start_idx_internal_fr(t = processed['fr t'], sig = channel_signal, n_packets = 32) for channel_signal in processed['fr data']]

#print(reset_indices)
#print(len(reset_indices))
#print(np.median(reset_indices))
#print(np.mean(reset_indices))

print(reset_indices)

reset_idx = int(np.nanmedian(reset_indices)+2)
print(reset_idx)

#for i in range(len(processed['fr data'])):
#    fig, ax = plt.subplots(1)
#    ax.plot(processed['fr t'], processed['fr data'][i], '.-')
#    ax.plot(processed['fr t'][reset_idx], processed['fr data'][i][reset_idx])




[15, 17, 19, 39, 19, 34, 20, 28, 19, 34, 29, 2, 45, 31, 37, 19, 40, 24, 5, 39, 29, 39, 21, 2, 40, 34, 5, 24, 2, 31, 34, 36, 7, 1, 20, 41, 8, 2, 40, 17, 40, 2, 42, 23, 44, 26, 34, 4, 28, 35, 36, 40, 4, 27, 2, 19, 8, 27, 27, 35, 16, 4, 37, 24, 41, 19, 19, 29, 2, 2, 36, 36, 36, 36, 2, 21, 2, 45, 45, 34, 16, 29, 2, 35, 2, 34, 2, 36, 8, 6, 2, 36, 41, 19, 25, 2, 29, 29, 2, 36, 40, 29, 2, 16, 40, 2, 40, 2, 16, 43, 7, 19, 2, 3, 2, 24, 34, 2, 27, 28, 2, 31, 5, 1, 34, 10, 26, 37, 2, 44, 40, 40, 35, 40, 8, 2, 34, 46, 29, 30, 10, 1, 39, 19, 10, 2, 2, 7, 37, 34, 29, 34, 29, 24, 29, 19, 2, 33, 27, 32, 2, 29, 1, 19, 29, 24, 2, 29, 42, 5, 30, 19, 17, 0, 34, 33, 40, 5, 34, 34, 7, 32, 34, 7, 5, 18, 34, 8, 19, 34, 45, 43, 40, 33, 19, 2, 6, 40, 34, 0, 16, 19, 0, 30, 39, 6, 43, 32, 2, 42, 16, 4, 18, 41, 21, 20, 14, 20, 45, 3, 44, 25, 26, 5, 44, 37, 32, 35, 15, 23, 11]
27


In [150]:
print(processed['start index'])

5


In [64]:
fig, ax = plt.subplots(1)

for i in range(len(processed['demod data'])):
    ax.plot(processed['demod t'], processed['demod data'][i] - np.average(processed['demod data'][i]) + 0.1*i,'.-')

In [36]:
for i in [1]:#range(len(processed['demod data'])):
    plt.plot(processed['fr data'][i])# - np.average(processed['fr data'][i]) + i*0.01)
    

In [119]:
print(processed['start index'])

27


In [51]:
print(processed['fr t'])

[0.000000e+00 2.048000e-03 4.096000e-03 ... 9.988096e+00 9.990144e+00
 9.992192e+00]


In [78]:
imp.reload(dm)

%matplotlib qt

fig, ax = plt.subplots(1)

for i in range(len(processed['fr data'])):

    t, data = dm.demodulate_with_template(t=processed['fr t'], sig = processed['fr data'][i], n_packets = 32, correct_phase_jumps = True, phase_jump_threshold=0.4)#, start_idx=32)

    ax.plot(t[:len(data)],data-np.average(data)+0.1*i)